In [ ]:
import modelscripts
import planetengine

system = modelscripts.isovisc_systemscript.build(res = 4)
planetengine.initials.apply({'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}, system)
system.solve()

import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nProcs = comm.Get_size()

def get_scales(variable):
    try:
        array = variable.data
    except:
        array = variable
    dims = array.shape[1]
    mins = np.array([np.min(array[:, dim]) for dim in range(dims)])
    allMins = np.min(np.array(comm.allgather(mins)), axis = 0)
    maxs = np.array([np.max(array[:, dim]) for dim in range(dims)])
    allMaxs = np.max(np.array(comm.allgather(maxs)), axis = 0)
    scales = np.dstack([allMins, allMaxs])[0]
    return scales

print(get_scales(system.velocityField))